# Descarga y preparación del entorno

## Instalación de dependencias

In [ ]:
! pip install -r requirements.txt   

## Descomprimir datos
En el repositorio se incluye un archivo comprimido con los datasets en formato `.csv` de los créditos presupuestarios utilizados en el trabajo.

Estos fueron descargados el 5/10/2024 de la página de [presupuesto abierto](https://www.presupuestoabierto.gob.ar/sici/datos-abiertos) del gobierno nacional argentino. Los archivos utilizados se encuentran en la página como "Presupuesto de gastos y su ejecución detallada - agrupación anual".

También se incluye un script de python para descargar todos los datasets de este tipo disponibles al momento (desde 1995 hasta 2024). Se puede encontrar en `utils/descarga-presupuestos.py`.

In [2]:
import zipfile
import os

# Cambiar al directorio "credito-anual"
os.chdir('credito-anual')

# Descomprimir el archivo "credito-anual.zip"
with zipfile.ZipFile('credito-anual.zip', 'r') as zip_ref:
    zip_ref.extractall()
    
os.chdir('..')

print('Descomprimido credito-anual.zip')

Descomprimido credito-anual.zip


# Manejo de los datos con Pandas

## Carga de .csv a DataFrames de Pandas

In [77]:
import pandas as pd
DFs = {}
for year in range(2012,2020):
  df = pd.read_csv(f'credito-anual/credito-anual-{year}.csv', decimal=',')
  df.name = f'{year}'
  DFs[year] = df

## Funciones para el procesamiento de datos

In [78]:
def generar_dataframe_periodo(DFs, start_year, end_year):
        df = pd.concat([DFs[year] for year in range(start_year, end_year + 1)], ignore_index=True)
        df.name = f'{start_year}-{end_year}'
        return df

def calcular_total_pagado(df):
        return df['credito_pagado'].sum()

def calcular_total_pagado_por_funcion(df, funcion):
        return df[df['funcion_desc'].str.contains(funcion, na=False)]['credito_pagado'].sum()

def imprimir_resultados(df, total_pagado, total_pagado_trabajo, porcentaje):
        print(f'{df.name}:')
        print(f'    Total pagado: {total_pagado:,.2f}')
        print(f'    Total pagado con función \'Trabajo\': {total_pagado_trabajo:,.2f}')
        print(f'    Porcentaje pagado con función \'Trabajo\': {porcentaje:.2%}')

## Calculo de inversión con función 'Trabajo' por períodos estudiados

In [79]:
periods = {}
porcentaje_por_periodo = {}
   
for period in ['2012-2015', '2016-2019', '2012-2019']:
        start_year, end_year = period.split('-')
        periods[period] = generar_dataframe_periodo(DFs, int(start_year), int(end_year))
        
        total_pagado = calcular_total_pagado(periods[period])
        total_pagado_trabajo = calcular_total_pagado_por_funcion(periods[period], 'Trabajo')
        porcentaje_por_periodo[period] = total_pagado_trabajo / total_pagado
        
        imprimir_resultados(periods[period], total_pagado , total_pagado_trabajo, porcentaje_por_periodo[period])

2012-2015:
    Total pagado: 3,673,561.01
    Total pagado con función 'Trabajo': 17,070.08
    Porcentaje pagado con función 'Trabajo': 0.46%
2016-2019:
    Total pagado: 12,317,476.66
    Total pagado con función 'Trabajo': 33,300.60
    Porcentaje pagado con función 'Trabajo': 0.27%
2012-2019:
    Total pagado: 15,991,037.68
    Total pagado con función 'Trabajo': 50,370.68
    Porcentaje pagado con función 'Trabajo': 0.31%


## Calculo de inversión con funcion 'Trabajo' por año

In [80]:
porcentaje_por_año = {}

for year in range(2012,2020):
    total_pagado = calcular_total_pagado(DFs[year])
    total_trabajo = calcular_total_pagado_por_funcion(DFs[year], 'Trabajo')
    porcentaje_por_año[year] = total_trabajo/total_pagado
    
    imprimir_resultados(DFs[year], total_pagado, total_trabajo, porcentaje_por_año[year])

2012:
    Total pagado: 531,870.32
    Total pagado con función 'Trabajo': 3,106.80
    Porcentaje pagado con función 'Trabajo': 0.58%
2013:
    Total pagado: 701,981.01
    Total pagado con función 'Trabajo': 3,507.28
    Porcentaje pagado con función 'Trabajo': 0.50%
2014:
    Total pagado: 1,081,679.25
    Total pagado con función 'Trabajo': 4,211.20
    Porcentaje pagado con función 'Trabajo': 0.39%
2015:
    Total pagado: 1,358,030.44
    Total pagado con función 'Trabajo': 6,244.80
    Porcentaje pagado con función 'Trabajo': 0.46%
2016:
    Total pagado: 2,054,865.50
    Total pagado con función 'Trabajo': 6,358.43
    Porcentaje pagado con función 'Trabajo': 0.31%
2017:
    Total pagado: 2,482,120.55
    Total pagado con función 'Trabajo': 9,735.44
    Porcentaje pagado con función 'Trabajo': 0.39%
2018:
    Total pagado: 3,193,254.13
    Total pagado con función 'Trabajo': 7,207.43
    Porcentaje pagado con función 'Trabajo': 0.23%
2019:
    Total pagado: 4,587,236.48
    Tota